In [0]:
external_azure_path = 'abfss://container20260122@datalake20260121.dfs.core.windows.net/'

dbutils.fs.ls(f'{external_azure_path}/01-bronze')

In [0]:
# Creating tmp tables to load product data into silver layer
allowed_tables = frozenset({'customers/'})

for delta_table in dbutils.fs.ls(f'{external_azure_path}/01-bronze'):
    if delta_table.name in allowed_tables:
        (
            spark.read
                .format("delta")
                .load(delta_table.path)
                .createOrReplaceTempView(f'tmp_bronze_{delta_table.name.rstrip('/')}')
        )

In [0]:
%sql
DESCRIBE tmp_bronze_customers

In [0]:
%py
df_silver_orders = spark.sql('''
    -- Creating SQL query to aggregate customer data
    SELECT
        customer_id
      , first_name
      , last_name
      , phone
      , email
      , street
      , city
      , state
      , zip_code
    FROM
      tmp_bronze_customers
    WHERE
      phone     IS NOT NULL
      AND phone NOT IN ('NULL', 'NULL ')
      AND email IS NOT NULL
      AND email NOT IN ('NULL', 'NULL ')  
''')

display(df_silver_orders)

In [0]:
# Writing on silver azure external storage in delta format
(df_silver_orders.write
    .mode('overwrite')
    .format('delta')
    .option('mergeSchema', 'true')
    .save(f'{external_azure_path}/02-silver/customers')
)

In [0]:
%sql
-- Creating a new schema to store silver tables
CREATE SCHEMA IF NOT EXISTS bike_store.silver_schema

In [0]:
%sql
DROP TABLE IF EXISTS bike_store.silver_schema.customers

In [0]:
%sql
-- Creating databricks delta table
CREATE TABLE IF NOT EXISTS bike_store.silver_schema.customers
LOCATION 'abfss://container20260122@datalake20260121.dfs.core.windows.net/02-silver/customers'